In [157]:
import time
import logging
import random
import datetime as dt

import requests
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [158]:
api_key = "X3LIHYNX5YEDL5SA"
symbol = "ETH"
market = "USD"
seq = 240
batch = 64
hidden = 128
layers = 4
dropout = 0.2
epochs = 60
lr = 3e-4
patience = 7
forecast_days = 14
seed = 2005

In [159]:
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [160]:
url = "https://www.alphavantage.co/query"
params = {
    "function": "DIGITAL_CURRENCY_DAILY",
    "symbol": symbol,
    "market": market,
    "apikey": api_key,
}

for attempt in range(5):
    r = requests.get(url, params=params, timeout=30)
    if r.status_code == 200 and "Time Series (Digital Currency Daily)" in r.json():
        raw = r.json(); break
    wait = 5 * 2 ** attempt
    logging.warning("HTTP %d → retry in %ds", r.status_code, wait)
    time.sleep(wait)

key = "Time Series (Digital Currency Daily)"
df = (pd.DataFrame.from_dict(raw[key], orient="index")
        .rename(columns=lambda c: c.split(". ", 1)[1])
        .astype(float)
        .sort_index())
df.index = pd.to_datetime(df.index)
close_series = df["close"]
logging.info("Loaded %d daily points", len(close_series))

In [161]:
scaler = MinMaxScaler()
prices = scaler.fit_transform(close_series.values.reshape(-1, 1)).astype(np.float32)
prices_flat = prices.squeeze(-1)

windows = sliding_window_view(prices_flat, seq + 1)
X = windows[:, :-1]
y = windows[:, -1:]


X = X[..., np.newaxis]
y = y[..., np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False, random_state=seed)

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)),
                          batch_size=batch, shuffle=True)

test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test)),
                         batch_size=batch)

In [162]:
class GRUForecast(nn.Module):
    def __init__(self, hidden_size=hidden, n_layers=layers, drop=dropout):
        super().__init__()
        self.gru = nn.GRU(input_size=1,
                          hidden_size=hidden_size,
                          num_layers=n_layers,
                          batch_first=True,
                          bidirectional=True,
                          dropout=drop if n_layers > 1 else 0.0)
        self.norm = nn.LayerNorm(hidden_size * 2)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self._init_weights()

    def _init_weights(self):
        for name, param in self.named_parameters():
            if "weight" in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif "bias" in name:
                nn.init.constant_(param, 0)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.norm(out[:, -1])
        return self.fc(out)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=3, verbose=True)

/Users/daniil/ml_python_3.11.11/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [163]:
best_rmse = float("inf")
wait_epochs = 0
best_state = None

for epoch in range(1, epochs + 1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d}")
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.squeeze(1).to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward(); optimizer.step()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    # evaluation
    model.eval(); se = 0.0; n = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb.to(device)).cpu()
            se += ((preds - yb.squeeze(1)) ** 2).sum().item(); n += len(yb)
    rmse = (se / n) ** 0.5
    logging.info("Epoch %02d | RMSE = %.4f", epoch, rmse)
    scheduler.step(rmse)

    if rmse + 1e-6 < best_rmse:
        best_rmse = rmse
        wait_epochs = 0
        best_state = model.state_dict()
    else:
        wait_epochs += 1
        if wait_epochs >= patience:
            logging.info("Early stopping triggered at epoch %d", epoch)
            break

Epoch 01:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 02:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 03:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 04:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 05:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 13:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 14:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 15:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 16:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 17:   0%|          | 0/2 [00:00<?, ?it/s]

In [164]:
model.eval()
window = torch.from_numpy(prices_flat[-seq:]).unsqueeze(0).unsqueeze(-1).to(device)
forecast = []
with torch.no_grad():
    for _ in range(forecast_days):
        norm_pred = model(window).item()
        price_pred = scaler.inverse_transform([[norm_pred]])[0, 0]
        forecast.append(price_pred)
        window = window.roll(-1, dims=1)
        window[0, -1, 0] = norm_pred

dates = [close_series.index[-1] + dt.timedelta(days=i + 1) for i in range(forecast_days)]
out_df = pd.DataFrame({"date": dates, "eth_usd_pred": forecast})

print(out_df)

         date  eth_usd_pred
0  2025-05-23   2799.795746
1  2025-05-24   2826.893856
2  2025-05-25   2844.524587
3  2025-05-26   2853.086808
4  2025-05-27   2855.863021
5  2025-05-28   2855.342830
6  2025-05-29   2853.244542
7  2025-05-30   2850.629386
8  2025-05-31   2848.069680
9  2025-06-01   2845.820909
10 2025-06-02   2843.946833
11 2025-06-03   2842.407866
12 2025-06-04   2841.134661
13 2025-06-05   2840.058472
